In [9]:
import pandas as pd

In [10]:
df_tornado_withdraw = pd.read_csv("../data/tornado_deposit.csv")[:90818]
df_tornado_deposit = pd.read_csv("../data/tornado_withdraw.csv")[:90818]

In [11]:
df_total = pd.concat([df_tornado_deposit, df_tornado_withdraw], ignore_index=True)
df_tornado_address = set(df_total['address'].unique().tolist())
print("Overall distinct address number: {}".format(len(df_tornado_address)))

Overall distinct address number: 53886


In [18]:
print(len(df_total["txhash"]))

181636


In [12]:
h_exact_match = pd.read_csv("../proceed/exact_match.csv")
h_exact_match["reason"] = "exact match"
h_gas_price = pd.read_csv("../proceed/gas_price.csv")
h_gas_price["reason"] = "gas price"
h_linked_tx = pd.read_csv("../proceed/linked_tx.csv")
h_linked_tx["reason"] = "linked tx"
h_same_num_tx = pd.read_csv("../proceed/same_num_tx.csv")
h_same_num_tx["reason"] = "same num tx"
h_exact_match.head()

,txhash,addr,clusterID,reason
0,0x14281b8f9287ca34c5019a40ca0fc8672225dd75e26e...,0x42f4aed51d679684df6e44666bf2fa5840769161,1533,exact match
1,0xdc96407fb9e4eec13aa6ea96c741a81e02ead56b2560...,0x870ce4cbe75814672c0f1e62d3d19474dbca2edc,2456,exact match
2,0x24fc47adde3475d923422e954f5e8db3cf769ba04367...,0xaf301de836c81deb8dff9dc22745e23c476155b2,1478,exact match
3,0x6e6890a19ec8fdb43d9e47736256a8249065af7e22b7...,0x16e54b35d789832440ab47ae765e6a8098280676,648,exact match
4,0x13ca5719e863542bcc48d5bf3d951452761aec547b6f...,0x2647a18deb7153deb82a4488f66180f7688b97e2,2966,exact match


In [13]:
print(len(h_exact_match["addr"].unique().tolist()))
print(len(h_gas_price["addr"].unique().tolist()))
print(len(h_linked_tx["addr"].unique().tolist()))
print(len(h_same_num_tx["addr"].unique().tolist()))

2660
1215
6065
2321


In [14]:
df_exploit = pd.concat([h_exact_match, h_gas_price, h_linked_tx, h_same_num_tx], ignore_index=True)
df_exploit = df_exploit.drop_duplicates(subset=['txhash','addr'], ignore_index=True)
exploit_address = set(df_exploit['addr'].unique().tolist())
print("Overall exploit distinct address number: {}".format(len(exploit_address)))


Overall exploit distinct address number: 10363


In [20]:
print(len(df_exploit["txhash"].unique().tolist()))

66889


In [15]:
def is_exploit(addr:set) -> bool:
    return addr <= exploit_address

def participate_tornado(addr: set) -> bool:
    return addr <= df_tornado_address

In [16]:
pairs_df = pd.read_csv("../data/all_ens_pairs.csv")
pairs_df = pairs_df.loc[pairs_df['address'].isin(set(df_total["address"].unique().tolist()))]
ground_truth = pairs_df.groupby("name").filter(lambda x: x['address'].count()>1)
ground_truth = ground_truth.groupby('name')["address"].agg(set).reset_index()
ground_truth["exploit"] = ground_truth["address"].apply(is_exploit)
ground_truth["is_tornado"] = ground_truth["address"].apply(participate_tornado)


In [17]:
len(ground_truth[ground_truth["is_tornado"]==True]), len(ground_truth[ground_truth["exploit"]==True])

(20, 4)